In [4]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import json
import dataset
import credentials
import sqlite3
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pickle
from Naive_bayes_model import *
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
f = open('nb1.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [24]:
f1 = open('rfp', 'rb')
RFClassifier = pickle.load(f1)
f1.close()

In [25]:
conn = sqlite3.connect('tweet2.db')
c = conn.cursor()

In [26]:
def pred(data):
    
    tweetProcessor = PreProcessTweets()
    preprocessedTrainingSet = tweetProcessor.processTweets(data)
    
    word_features = buildVocabulary(preprocessedTrainingSet)
#     print("\n",word_features,"\n")
#     buildVocabulary(preprocessedTrainingSet)
    trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
    
    print(preprocessedTrainingSet[0][0])
    
    NB = NBayesClassifier.classify(extract_features(preprocessedTrainingSet[0][0],word_features))
    
    return NB
    

In [27]:

def attributeselection(text,query_word):
    attr= []

    #text = "Alleged East Bay serial arsonist arrested #SanFrancisco - http://t.co/ojuHfkHVb2"
    text = nltk.word_tokenize(text)
    poslist = nltk.pos_tag(text)
#     print(poslist)

    verblist = []
    for i in range(len(poslist)):
        if poslist[i][1].startswith('V'):
            verblist.append(poslist[i])
    
    word_count = len(poslist)
#     print(word_count)

    if len(verblist) == 0:
        l = (query_word,'VB')
        verblist.append(l)
    
#     print(verblist,"\n")
    verb_count = len(verblist)
#     print(verb_count)

   # query_word = 'arson'

    words_before = words_after = 0
    for i in range(len(poslist)):
        if poslist[i][0].startswith(query_word):
            words_before = i

    words_after = word_count - words_before - 1

#     print(words_before)
#     print(words_after)
    verbs = []    
    attr.append(word_count)
#     for i in verblist:
#         print(i[0])
#         verbs.append(i[0])
#     attr.append(verbs)
    attr.append(verb_count)
    attr.append(words_before + 1)
#     attr_query_word.append(query_word)
    attr.append(words_before)
    attr.append(words_after)
    
    
        
    column_names = ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
    print(attr,column_names)
    df = pd.DataFrame([attr], columns=column_names)
    arr_x=df.to_numpy(dtype=object)
    print(arr_x,arr_x.shape)
    y_pred = RFClassifier.predict(arr_x)
    print("PRIORITY: ",y_pred)
    
    

In [28]:
class TwitterStreamer():
    
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
        
        stream = Stream(auth, listener,tweet_mode='extended')
        
        stream.filter(languages=["en"],track=['coronavirus'])
        
 
class StdOutListener(StreamListener):
    
    def on_status(self, status):
        print(status.text)

    def on_error(self, status):
        print(status)

    def on_data(self, data):
        all_data             = json.loads(data)
        created_at           = all_data['created_at']
        favorite_count       = all_data['favorite_count']
        favorited            = all_data['favorited']
        filter_level         = all_data['filter_level']
        lang                 = all_data['lang']
        retweet_count        = all_data['retweet_count']
        retweeted            = all_data['retweeted']
        source               = all_data['source']
        text                 = all_data['text']
        truncated            = all_data['truncated']
        user_created_at      = all_data['user']['created_at']
        user_followers_count = all_data['user']['followers_count']
        user_location        = all_data['user']['location']
        user_lang            = all_data['user']['lang']
        user_name            = all_data['user']['name']
        user_screen_name     = all_data['user']['screen_name']
        user_time_zone       = all_data['user']['time_zone']
        user_utc_offset      = all_data['user']['utc_offset']
        user_friends_count   = all_data['user']['friends_count']
        geo_enabled          = all_data['user']['geo_enabled']  
        coordinates          = all_data['coordinates']
        geo                  = all_data['geo']
        place                = all_data['place']
        if place is not None:
            place_name       = all_data['place']['name']
        else:
            place_name       = all_data['place']

        print("Tweet: "+text+"\n")
        
        data = [{'text': text,'Classfication': ''}]
        
        info = pred(data)
        
        if info == 'Relevant' or info == 1 or info == '1':
            attributeselection(text,'coronavirus')
            
        print("\n Class: "+info+"\n")
                         
        c.execute('''INSERT INTO tweets 
        (created_at, favorite_count, favorited, filter_level, lang, 
                         retweet_count, retweeted, source, text, truncated, user_created_at,  
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ) 
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
            (created_at, favorite_count, favorited, filter_level, lang, retweet_count, 
                         retweeted, source, text, truncated, user_created_at, 
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ))
        
        c.execute('''INSERT INTO tweet_class 
        (text, class) 
            VALUES (?,?)''', 
            (text, info))
        
        conn.commit()

In [29]:
if __name__ == '__main__':
    
    hash_tag_list = ['coronavirus']
    fetched_tweets_filename = "tweets.json"
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

Tweet: Looks bad

['looks', 'bad']

 Class: 0

Tweet: ETF Blocks
📈  $VXX  🧨 #Volatility
📊  5,000,000  😱
💵  59.56
💚  +0.79 | +1.33%

Largest I have ever seen. Not so sure… https://t.co/2e0HMAEPqE

['etf', 'blocks', '📈', 'vxx', '\U0001f9e8', 'volatility', '📊', '5,000,000', '😱', '💵', '59.56', '💚', '+0.79', '+1.33', 'largest', 'ever', 'seen', 'sure…']

 Class: 0

Tweet: RT @DearDean22: He told wealthy friends. Then cashed out a cool $1.4m in stock. Then voted AGAINST the covid-19 relief bill. He needs to go

['rt', 'told', 'wealthy', 'friends', 'cashed', 'cool', '1.4m', 'stock', 'voted', 'covid-19', 'relief', 'bill', 'needs', 'go']

 Class: 0

Tweet: RT @UlsterExec: Thank you @SenSchumer for your leadership passing an emergency legislative package to combat #COVID19. This bill will provi…

['rt', 'thank', 'leadership', 'passing', 'emergency', 'legislative', 'package', 'combat', 'covid19', 'bill', 'provi…']

 Class: 0

Tweet: RT @DrDooleyMD: I just received notice that one of my patients is

Tweet: RT @evebennettx: We are literally living in a 2050 GCSE History question

['rt', 'literally', 'living', '2050', 'gcse', 'history', 'question']
[14, 2, 1, 0, 13] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[14 2 1 0 13]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: RT @JoyceWhiteVance: 1/2 Bureau of Prisons, state prisons, local jails all need plans for Coronavirus. They should already be in place &amp; fu…

['rt', '1/2', 'bureau', 'prisons', 'state', 'prisons', 'local', 'jails', 'need', 'plans', 'coronavirus', 'already', 'place', 'amp', 'fu…']
[30, 2, 1, 0, 29] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[30 2 1 0 29]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: @MarijuanatimesX You can’t write about weed from home?
I’m smoking it from home just fine.
Maybe, do a however many… https://t.co/LJ3dAzpVBl

['’', 'write', 'weed', 'home', '’', 'smoking', 'home', 'fine', 'maybe', 'however', 'many…']

 Class: 0

Tweet: RT @Sm

PRIORITY:  [0]

 Class: 1

Tweet: RT @241Latina: Remember this when you take more than you need! 😢💔💔💔
(Wish I could take her home with me) 😢 #Coronavirus https://t.co/o7psZW…

['rt', 'remember', 'take', 'need', '😢💔💔💔', 'wish', 'could', 'take', 'home', '😢', 'coronavirus']

 Class: 0

Tweet: RT @noonanjo: At home test kits available Mon. NY now testing more than China, S Korea.

Military released 2k ventilators, Admin announced…

['rt', 'home', 'test', 'kits', 'available', 'mon', 'ny', 'testing', 'china', 'korea', 'military', 'released', '2k', 'ventilators', 'admin', 'announced…']
[28, 2, 1, 0, 27] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[28 2 1 0 27]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: RT @nnboogie: And this...ladies and gentlemen is how citizens of the UK have been spreading #coronavirus all over the Caribbean and elsewhe…

['rt', '...', 'ladies', 'gentlemen', 'citizens', 'uk', 'spreading', 'coronavirus', 'caribbean', 'elsewhe…']
[27, 5, 21,

KeyError: 'created_at'